In [1]:
using Pkg; for p in ("Gym","Plots"); haskey(Pkg.installed(),p) || Pkg.add(p); end
Pkg.add("PyCall")
ENV["GYM_ENVS"]="atari:classic_control:box2d"
ENV["PYTHON"]="/usr/local/bin/python3"
Pkg.build("PyCall"); Pkg.build("Gym")
using Gym, PyCall, Knet, Base.Iterators
np = pyimport("numpy")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
  Building Conda ─→ `~/.julia/packages/Conda/kLXeC/deps/build.log`
  Building PyCall → `~/.julia/packages/PyCall/ttONZ/deps/build.log`
  Building Conda ─→ `~/.julia/packages/Conda/kLXeC/deps/build.log`
  Building PyCall → `~/.julia/packages/PyCall/ttONZ/deps/build.log`
  Building Gym ───→ `~/.julia/packages/Gym/QZ2GY/deps/build.log`


PyObject <module 'numpy' from '/usr/local/lib/python3.7/site-packages/numpy/__init__.py'>

In [2]:
Pkg.add("Images"); using Images; Pkg.add("Colors"); using Colors
Pkg.add("ImageTransformations"); using ImageTransformations

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [3]:
mutable struct Linear; w; b; end

function Linear(inputsize::Int, outputsize::Int)
    w = param(outputsize, inputsize)
    b = param0(outputsize)
    Linear(w,b)
end
function (l::Linear)(x)
    l.w * x .+ l.b
end

In [4]:
mutable struct Projection; w; end

function Projection(inputsize::Int, outputsize::Int)
    w = param(outputsize, inputsize)
    Projection(w)
end
function (p::Projection)(x)
    p.w * x
end

In [5]:
function mse_loss(arr1, arr2)
    sum((arr1'-arr2).^2)/length(arr1)
end

mse_loss (generic function with 1 method)

In [43]:
mutable struct DQN
    fc1
    fc2
    output
end

function DQN(img_height::Int, img_width::Int, input_channels::Int, hidden_size1::Int, hidden_size2::Int, action_space_size::Int)
    fc1 = Linear(img_height*img_width*input_channels, hidden_size1)
    fc2 = Linear(hidden_size1, hidden_size2)
    output = Projection(hidden_size2, action_space_size)    
    DQN(fc1, fc2, output)
end

function (model::DQN)(x)
    model.output(relu.(model.fc2(relu.(model.fc1(x)))))
end


function (model::DQN)(x, indexes, q_star)
    mse_loss(model.output(relu.(model.fc2(relu.(model.fc1(x)))))[indexes],q_star)
end

In [7]:
mutable struct ReplayMemory
    capacity
    memory
    push_count
    batchsize
end


function ReplayMemory(capacity::Int, batchsize::Int)
    push_count = 0
    memory = []
    ReplayMemory(capacity, memory, push_count, batchsize)
end


function PushToReplayMemory!(r::ReplayMemory, e)
    r.push_count += 1
    if(length(r.memory) < r.capacity)
        push!(r.memory, e)
    else
        r.memory[r.push_count%r.capacity] = e
    end
    
end

function CanProvideSample(r::ReplayMemory)
    return r.push_count >= r.batchsize
end

function Sample(r::ReplayMemory)
    return r.memory[rand(1:min(r.push_count, r.capacity),r.batchsize)]
end

Sample (generic function with 1 method)

In [8]:
struct EpsilonGreedyStrategy
    max
    min
    decay
end

function GetExplorationRate(e::EpsilonGreedyStrategy, current_step::Int)
    e.min + (e.max-e.min) * exp(-1*current_step*e.decay)
end

GetExplorationRate (generic function with 1 method)

In [44]:
mutable struct Agent
    current_step
    strategy
    num_actions
end

function Agent(strategy::EpsilonGreedyStrategy, num_actions::Int)
    current_step = 0
    return Agent(current_step, strategy, num_actions)
end

function GetAction(a::Agent, state, policy_net, env)
    a.current_step += 1
    ϵ = GetExplorationRate(a.strategy, a.current_step)
    
    if(ϵ > rand())
        return rand(0:env.action_space.n-1)
    else
        return argmax(policy_net(state))-1
    end 
end

GetAction (generic function with 1 method)

In [58]:
reward = 0
num_episodes = 1000
batch_size = 25 #
γ = 0.999
ϵ_start = 1
ϵ_decay = 0.001
ϵ_end = 0.01
target_update = 10
memory_size = 100000
lr = 0.1 #
image_height = 40 
image_width = 90 
input_channels = 1
hidden_size1 = 24
hidden_size2 = 32
moving_average_length = 1 #

1

In [11]:
cropfromtop = 0.4
cropfrombottom = 0.2
target_height = 40
target_width = 90

90

In [12]:
function GetPreProcessedImageArray(env, cropfromtop::Float64, cropfrombottom::Float64, target_height::Int, target_width::Int)
    state_pyobject = np.ascontiguousarray(render(env, mode="rgb_array"), dtype = np.float64)/255
    height, width, channels = size(state_pyobject)
    state_image = permutedims(state_pyobject,(3,1,2))
    state_image2 = (state_image[:,Int(height*cropfromtop):Int((1-cropfrombottom)*height)-1,:])
    grayscale = Gray.(state_image2)
    reshape(Float64.(imresize(grayscale, (1,target_height, target_width))), target_width*target_height)
end
    

GetPreProcessedImageArray (generic function with 1 method)

In [13]:
function extract_tensors(experiences, inputsize, batchsize)
    tensors = reshape(collect(flatten(experiences)), (4,batchsize))
    states = reshape(collect(flatten(tensors[1,:])), (inputsize, batchsize))
    actions = tensors[2,:]
    next_states = reshape(collect(flatten(tensors[3,:])), (inputsize, batchsize))
    rewards = tensors[4,:]
    return states, actions, next_states, rewards
end

extract_tensors (generic function with 1 method)

In [14]:
function GetMovingAverage(collection, n)
    len = length(collection)
    if(n>len)
        return 0
    else
        return sum(collection[len-n+1:end])/n
    end
end

GetMovingAverage (generic function with 1 method)

In [59]:
env = GymEnv("CartPole-v1")
episode_rewards = []
replay_memory = ReplayMemory(memory_size, batch_size)
policy_network = DQN(image_height, image_width, input_channels, hidden_size1, hidden_size2, env.action_space.n)
target_network = deepcopy(policy_network)
strategy = EpsilonGreedyStrategy(ϵ_start, ϵ_end, ϵ_decay)
agent = Agent(strategy, env.action_space.n)
total_reward_per_episode = 0

for i = 1:num_episodes
    done = false
    reset!(env) 
    current_state = zeros(target_width*target_height)
    println("episode: ", i, " 100MA: ", GetMovingAverage(episode_rewards,moving_average_length))
    total_reward_per_episode = 0
    time_step = 0
    while(!done) 
        time_step += 1
        action = GetAction(agent, current_state, policy_network, env)
        observation, reward, done, information = step!(env, action) 
        if(done)
            next_state = zeros(target_width*target_height)
        else
            next_state = GetPreProcessedImageArray(env, cropfromtop, cropfrombottom, target_height, target_width) - current_state
        end
        
        total_reward_per_episode += reward
        PushToReplayMemory!(replay_memory, [current_state, action, next_state, reward])
        current_state = next_state
        if(CanProvideSample(replay_memory))
            experiences = Sample(replay_memory)
            states, actions, next_states, rewards = extract_tensors(experiences, image_height*image_width*input_channels, batch_size)
            indexes = collect(0:batch_size-1)*env.action_space.n + actions .+ 1
            q_star = maximum(target_network(next_states), dims=1) * γ + rewards'  
            loss = @diff mse_loss(policy_network(states)[indexes], q_star)
        
            println("before: ", loss)
            #for p in params(loss)
            #    p = p - lr * grad(loss, p)
            #end
            
            before = sum(policy_network.fc1.w)
            tup = (states, indexes, q_star)
            list = [tup]
            adam!(policy_network, take(cycle(list),1))
            #progress!(a)
            println("after: ", mse_loss(policy_network(states)[indexes], q_star))
            after = sum(policy_network.fc1.w)
            #println("before: ", before)
            #println("after: ", after)
        end
        
        if(done)
            push!(episode_rewards, total_reward_per_episode)
        end
        if(agent.current_step%target_update == 0)
            target_network = deepcopy(policy_network)
        end
    end
    
end

episode: 1 100MA: 0
episode: 2 100MA: 14.0
episode: 3 100MA: 9.0
before: T(1.1820595925540396)
after: 0.7999420127717701
before: T(0.8683968466423625)
after: 0.7412921750896868
before: T(0.6675459609243337)
after: 0.5875022966800014
before: T(0.8203835379414621)
after: 0.781307652659242
before: T(0.7585250559625671)
after: 0.7705175404456268
before: T(0.6152861973848964)
after: 0.6020382231761651
before: T(4.121925114053096)
after: 4.071208571672255
before: T(2.6087203834786656)
after: 2.561502421004492
before: T(3.2890496915648235)
after: 3.249753910007606
before: T(3.2877429752491567)
after: 3.2527718297476986
before: T(1.9881597671776348)
after: 1.9746614558579236
before: T(3.7624895801021485)
after: 3.753036408998357
before: T(1.8078382075471444)
after: 1.7908062336587987
before: T(2.517909897299717)
after: 2.502608983347966
before: T(2.4396388947881085)
after: 2.4202527949220554
before: T(2.156380136593468)
after: 2.149331304754159
before: T(1.8668134116929336)
after: 1.8668992476

InterruptException: InterruptException: